In [48]:
!pip install torch
!pip install sentence-transformers
!pip install transformers
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install pinecone
!pip install pinecone-client
!pip install langchain


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  Obtaining dependency information for pinecone-plugin-inference<2.0.0,>=1.0.3 from https://files.pythonhosted.org/packages/d8/5e/a7eb453cfb3aa9c8c995a1dca5fcf57f79b67400593d5c6759571567e30c/pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=15.0.0 from https://files.pythonhosted.org/packages/3a/2e/3b99f8a3d9e0ccae0e961978a0d0089b25fb46ebbcfb5ebae3cca179a5b3/pyarrow-18.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee34/dill-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/11/a7/81dba5010f7e733de88af9555725146fc133be97ce36533867f4c7e75066/xxhash-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for multiprocess<0.70.17 from https://files.pythonhosted.org/packages/0a/7d/a988f258104dc

In [1]:
import numpy as np
import pandas as pd
import os
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import math
import csv
import random
from openai import OpenAI


/home/maksim/PycharmProjects/Advanced-Information-Retrieval/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['OPENAI_API_KEY'] = 'api_key'

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [2]:
from pinecone import Pinecone, ServerlessSpec, Index

# Initialize Pinecone
pc = Pinecone(api_key="api_key")

index_name = "air"

# Check if the index exists; create it if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # Dimension of your embeddings
        metric="cosine",  # Similarity metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(index_name)

print("Pinecone setup complete!")


Pinecone setup complete!


In [4]:

def load_lyrics_dataset(file_path):
    try:
        # Attempt to read the file with 'latin1' encoding
        df = pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')
    except Exception as e:
        print(f"Failed to read file: {e}")
        return None

    # Preprocessing
    # Remove rows with missing values in key columns
    df = df.dropna(subset=['Artist', 'Song', 'Lyrics'])
    # Remove rows where lyrics have fewer than 3 words
    df['lyrics_word_count'] = df['Lyrics'].apply(lambda x: len(str(x).split()))
    df = df[df['lyrics_word_count'] >= 3]

    df = df.drop(columns=['lyrics_word_count'])
    return df



# def retrieve_top_k_songs_pinecone(query, index, bi_encoder, k=10):
#     query_embedding = bi_encoder.encode([query], convert_to_tensor=False)
#     results = index.query(vector=query_embedding.tolist(), top_k=k * 2, include_metadata=True)  # Fetch more to account for duplicates

#     # Use a set to keep track of unique songs
#     unique_songs = {}
#     for match in results.matches:
#         key = (match["metadata"]["Song"], match["metadata"]["Artist"])  # Unique identifier
#         if key not in unique_songs:
#             unique_songs[key] = {
#                 "Song": match["metadata"]["Song"],
#                 "Artist": match["metadata"]["Artist"],
#                 "Lyric": match["metadata"]["Lyric"],
#                 "score": match.score,
#             }

#         # Stop once we have the top k unique songs
#         if len(unique_songs) >= k:
#             break

#     # Sort by score and return top k unique songs
#     return sorted(unique_songs.values(), key=lambda x: x["score"], reverse=True)


def preprocess_lyrics(lyrics, min_segment_size=3, max_segments=10):
    words = lyrics.split()
    total_words = len(words)

    if total_words <= min_segment_size:
        return [lyrics]

    segment_size = max(min_segment_size, math.ceil(total_words / max_segments))
    segments = [" ".join(words[i:i+segment_size]) for i in range(0, total_words, segment_size)]
    return segments

def prepare_embedding_training_data(input_csv_path, output_csv_path, queries_per_song=5, window_size=10, step=5):
    df = pd.read_csv(input_csv_path, encoding='latin1')

    # Convert Non-String Values to Strings
    df["Lyrics"] = df["Lyrics"].fillna("").astype(str)

    # Verify required columns exist
    required_columns = ["Song", "Artist", "Lyrics"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Store query-lyric pairs with labels
    data = []

    for index, row in df.iterrows():
        full_lyrics = row["Lyrics"]

        # Use the row index as a unique song ID
        song_id = index

        # Split lyrics into words
        words = full_lyrics.split()

        # Generate positive queries (labels = 1.0)
        for start in range(0, len(words) - window_size + 1, step):
            query = " ".join(words[start:start + window_size])
            data.append({
                "query": query,
                "song_id": song_id,
                "full_lyrics": full_lyrics,
                "label": 1.0
            })

        # Generate negative queries (labels = 0.0)
        negative_samples = random.sample(df[df.index != index].to_dict(orient="records"), k=queries_per_song)
        for neg_sample in negative_samples:
            data.append({
                "query": query,
                "song_id": neg_sample["Song"],
                "full_lyrics": neg_sample["Lyrics"],
                "label": 0.0
            })

    # Save the prepared dataset as a CSV file
    processed_df = pd.DataFrame(data)
    processed_df.to_csv(output_csv_path, index=False)

    print(f"Dataset prepared and saved to {output_csv_path}")


In [4]:
prepare_embedding_training_data(input_csv_path="/home/maksim/PycharmProjects/Advanced-Information-Retrieval/data/dataset.csv", output_csv_path="/home/maksim/PycharmProjects/Advanced-Information-Retrieval/data/output_dataset.csv")


Dataset prepared and saved to /home/maksim/PycharmProjects/Advanced-Information-Retrieval/data/output_dataset.csv


In [70]:
finetune_embedding_model("/home/maksim/PycharmProjects/Advanced-Information-Retrieval/data/output_dataset.csv")


Map: 100%|██████████| 339317/339317 [00:21<00:00, 15609.77 examples/s]
/home/maksim/PycharmProjects/Advanced-Information-Retrieval/.venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_lyrics(lyrics, chunk_size=100, overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap
    )
    return splitter.split_text(lyrics)


In [13]:
def preprocess_and_store_embeddings(data, index, chunk_size=100, overlap=50, batch_size=100):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    bi_encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    rows = []

    for idx, row in data.iterrows():
        lyrics = row['Lyrics']
        Song = row['Song']
        artist = row['Artist']

        # Convert lyrics to string and handle potential NaN values
        lyrics = str(lyrics)  # Ensure lyrics is a string
        if lyrics.lower() == 'nan':
            continue

        # Use LangChain chunker
        chunks = splitter.split_text(lyrics)
        for i, chunk in enumerate(chunks):
            rows.append((f"{idx}-{i}", chunk, Song, artist))

    for i in range(0, len(rows), batch_size):
        batch = rows[i:i+batch_size]

        # Extract chunks for embedding
        chunks = [row[1] for row in batch]
        embeddings = bi_encoder.encode(chunks, convert_to_tensor=False)

        # Prepare data for upsert
        vectors = []
        for (vector_id, chunk, Song, artist), embedding in zip(batch, embeddings):
            metadata = {
                "Song": Song,
                "Artist": artist,
                "Lyrics": chunk
            }
            vectors.append((vector_id, embedding.tolist(), metadata))

        # Upsert the batch to Pinecone
        index.upsert(vectors)
        print(f"Upserted batch {i//batch_size + 1}/{(len(rows) + batch_size - 1) // batch_size}")

    print("Embeddings stored in Pinecone!")


In [26]:
class BiEncoder:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.model = SentenceTransformer(model_name)

    def encode_texts(self, texts):
        """Used for encoding lyrics into embeddings."""
        return self.model.encode(texts, convert_to_tensor=True, show_progress_bar=True)


In [22]:
def refine_query_with_chatgpt(query):
    # Few-shot examples to guide the model
    example_message = (
        "You are a helpful assistant tasked with extracting and correcting song lyrics "
        "from user input and identifying the author if mentioned. You should return only the "
        "lyric portion of the query and refine it for correctness. Format your output as follows:\n"
        "query: <start_query>original_query<end_query>\n"
        "refined_query: <start_refined_query>refined_lyric<end_refined_query>\n"
        "author: <start_author>author_name<end_author>\n"
        "If the author is not mentioned, return 'NOT_MENTIONED' for the author.\n\n"
        "Examples:\n"
        "1. User input: \"What's the song from Rihanna that goes like 'how about a round of applause'\"\n"
        "   Output:\n"
        "   query: <start_query>how about a round of applause<end_query>\n"
        "   refined_query: <start_refined_query>How about a round of applause<end_refined_query>\n"
        "   author: <start_author>Rihanna<end_author>\n\n"
        "2. User input: \"Twinkle twinkle litl star how I wondr wht u ar\"\n"
        "   Output:\n"
        "   query: <start_query>Twinkle twinkle litl star how I wondr wht u ar<end_query>\n"
        "   refined_query: <start_refined_query>Twinkle twinkle little star how I wonder what you are<end_refined_query>\n"
        "   author: <start_author>NOT_MENTIONED<end_author>\n\n"
        "3. User input: \"song about a broken heart\"\n"
        "   Output:\n"
        "   query: <start_query>a broken heart<end_query>\n"
        "   refined_query: <start_refined_query>A broken heart<end_refined_query>\n"
        "   author: <start_author>NOT_MENTIONED<end_author>\n\n"
        "Now process this input:\n"
        f"User input: {query}"
    )

    # Call the OpenAI API
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant tasked with extracting and correcting song lyrics from user input and identifying the author if mentioned."},
            {"role": "user", "content": example_message},
        ],
        model="gpt-4o",
    )

    # Extract and format the result
    output = chat_completion.choices[0].message.content.strip()

    # Manually parse the refined query from the output
    start_tag = "<start_refined_query>"
    end_tag = "<end_refined_query>"
    refined_query = ""
    if start_tag in output and end_tag in output:
        refined_query = output.split(start_tag)[-1].split(end_tag)[0].strip()

    print("Refined query:\n", refined_query)
    return refined_query


In [27]:
def song_retrieval_pipeline(query, index, bi_encoder_model='sentence-transformers/all-mpnet-base-v2',
                            cross_encoder_model='cross-encoder/ms-marco-MiniLM-L-6-v2',
                            bi_encoder_k=100, cross_encoder_k=10):
    # Bi-Encoder
    bi_encoder = BiEncoder(bi_encoder_model)

    refined_query = refine_query_with_chatgpt(query)
    query_embedding = bi_encoder.encode_texts([refined_query])[0].tolist()

    # Query Pinecone
    results = index.query(vector=query_embedding, top_k=bi_encoder_k, include_metadata=True)

    # Initial retrieval with duplicate removal
    candidates = []
    seen_songs = set()

    for match in results["matches"]:
        song_key = (match["metadata"]["Song"], match["metadata"]["Artist"])
        if song_key not in seen_songs:
            seen_songs.add(song_key)
            candidates.append({
                "track_name": match["metadata"]["Song"],
                "artist_name": match["metadata"]["Artist"],
                "lyrics_chunk": match["metadata"]["Lyrics"],
                "score": match.score
            })
            if len(candidates) >= bi_encoder_k:  # Stop if we have enough unique songs
                break

    print("Top 10 results before reranking:")
    for candidate in candidates[:10]:
        print(f"Track: {candidate['track_name']}, Artist: {candidate['artist_name']}, Score: {candidate['score']}")

    candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)[:cross_encoder_k]
    lyrics_chunks = [candidate["lyrics_chunk"] for candidate in candidates]
    from sentence_transformers import CrossEncoder
    # Cross-Encoder
    cross_encoder = CrossEncoder(model_name=cross_encoder_model)
    candidate_pairs = [(query, chunk) for chunk in lyrics_chunks]
    scores = cross_encoder.predict(candidate_pairs, batch_size=16)

    # Re-ranking
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    re_ranked_songs = [candidates[i] for i in ranked_indices]

    print("\nTop 10 results after reranking:")
    for song in re_ranked_songs[:10]:
        print(f"Track: {song['track_name']}, Artist: {song['artist_name']}, Score: {song['score']}")

    return re_ranked_songs

#song_retrieval_pipeline("something about an applouse", index)


In [28]:
# Main program
dataset_path = "/content/drive/MyDrive/Colab Notebooks/dataset.csv"
corpus_path = "/content/drive/MyDrive/Colab Notebooks/corpus.csv"
queries_path = "/content/drive/MyDrive/Colab Notebooks/queries.csv"
qrels_path = "/content/drive/MyDrive/Colab Notebooks/qrels.csv"
query = "songs about broken heart"

if not index.describe_index_stats()["total_vector_count"]:

    if os.path.exists(dataset_path):
        print("Dataset obtained.")
        data = load_lyrics_dataset(dataset_path)

        if not (os.path.exists(corpus_path) and os.path.exists(queries_path) and os.path.exists(qrels_path)):
            print("Required files not found. Generating fine-tuning datasets...")

            # Generate fine-tuning datasets
            queries, corpus = create_finetuning_dataset(data, num_queries=5, num_negative_pairs=100, qrels_path=qrels_path)
            preprocess_and_store_embeddings(data, index, chunk_size=100, overlap=50)  # Use chunking params

            # Save queries and corpus to CSV
            queries_df = pd.DataFrame(queries)
            corpus_df = pd.DataFrame(corpus)
            queries_df.to_csv(queries_path, index=False)
            corpus_df.to_csv(corpus_path, index=False)
            print("Datasets generated and saved.")
        else:
            print("Datasets already exist. Skipping dataset generation.")

results = song_retrieval_pipeline(query, index)
print("Top retrieved songs:")
for idx, song in enumerate(results):
    print(f"{idx + 1}. {song['track_name']} by {song['artist_name']}")
    print(f"Lyrics Chunk: {song['lyrics_chunk']}\n")




Refined query:
 Broken heart


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.69it/s]


Top 10 results before reranking:
Track: cross my broken heart, Artist: the jets, Score: 0.630817056
Track: brokenhearted, Artist: brandy featuring wanya morris, Score: 0.628032863
Track: two to make it right, Artist: seduction, Score: 0.61804378
Track: breakeven, Artist: the script, Score: 0.601431489
Track: luv u better, Artist: ll cool j, Score: 0.601056
Track: time love and tenderness, Artist: michael bolton, Score: 0.60098505
Track: total eclipse of the heart, Artist: nicki french, Score: 0.599912345
Track: im still in love with you, Artist: sean paul featuring sasha, Score: 0.598965168
Track: second chance, Artist: 38 special, Score: 0.598877311
Track: ill never fall in love again, Artist: tom jones, Score: 0.592734754

Top 10 results after reranking:
Track: cross my broken heart, Artist: the jets, Score: 0.630817056
Track: im still in love with you, Artist: sean paul featuring sasha, Score: 0.598965168
Track: time love and tenderness, Artist: michael bolton, Score: 0.60098505
Tra